In [ ]:
!pip install torch torchvision torchaudio
!pip install opencv-python scikit-image tqdm pyyaml


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Classification

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jocelyndumlao/malhari-dataset")

print("Path to dataset files:", path)

100%|██████████| 428M/428M [00:08<00:00, 52.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jocelyndumlao/malhari-dataset/versions/1


In [ ]:
# prompt: from my drive there's a folder called malhari dataset edited, open it and store it's path in root_dir variable

from google.colab import drive
import kagglehub
import os

!pip install torch torchvision torchaudio
!pip install opencv-python scikit-image tqdm pyyaml

drive.mount('/content/drive')
# ### Classification

# Download latest version
# path = kagglehub.dataset_download("jocelyndumlao/malhari-dataset")

# print("Path to dataset files:", path)

root_dir = "/content/drive/MyDrive/Malhari Dataset Edited"

# Check if the directory exists
if os.path.exists(root_dir):
    print(f"Root directory found: {root_dir}")
else:
    print(f"Error: Directory '{root_dir}' not found in your Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Root directory found: /content/drive/MyDrive/Malhari Dataset Edited


In [ ]:
!pip install Pillow
import os
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from torchvision.models import resnet34
import torch.optim as optim
from sklearn.model_selection import KFold

# prompt: in above code ['ASC_H', 'ASC_US', 'HSIL', 'LSIL', 'NILM'] this should have label as 0,1,2,3,4 respectively

# ... (Your existing code)

# Data paths and parameters
data_dir = root_dir
image_size = 224  # Adjust as needed
batch_size = 64
num_epochs = 30
learning_rate = 1e-4

# Define label mapping
label_mapping = {'HSIL': 0, 'LSIL': 1, 'NILM': 2, 'SSC': 3}

# Image transformations
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.RandomHorizontalFlip(),  # Random Flip Augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ... (Your existing code)

# Load data
all_images = []
all_labels = []

for class_name in ['LSIL', 'HSIL', 'NILM', 'SSC']:
    class_path = os.path.join(data_dir, class_name)
    for filename in os.listdir(class_path):
        # Construct the full image path directly
        image_path = os.path.join(class_path, filename)
        # Check if it's a file and if it has a valid image extension
        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.bmp')):
            img = Image.open(image_path).convert("RGB")
            img_transformed = transform(img)
            all_images.append(img_transformed)
            all_labels.append(label_mapping[class_name])

In [ ]:
all_images

In [ ]:


all_images = torch.stack(all_images)
all_labels = torch.tensor(all_labels)
# ... (Rest of your code)

tensor([0, 0, 0,  ..., 4, 4, 4])

In [ ]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Define device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Hyperparameters
batch_size = 64
num_epochs = 30
learning_rate = 1e-4

# 5-fold cross-validation
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

for fold, (train_ids, test_ids) in enumerate(kfold.split(all_images)):
    print(f"\n----- Fold {fold+1} -----")

    # Sample data for current fold
    train_images = torch.stack([all_images[i] for i in train_ids])
    train_labels = torch.tensor([all_labels[i] for i in train_ids])
    test_images = torch.stack([all_images[i] for i in test_ids])
    test_labels = torch.tensor([all_labels[i] for i in test_ids])

    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = torch.utils.data.TensorDataset(test_images, test_labels)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize DenseNet-121 model
    model = models.densenet121(pretrained=True)  # Load pretrained DenseNet-121
    num_ftrs = model.classifier.in_features
    model.classifier = nn.Linear(num_ftrs, 4)  # Modify classifier for 4 classes
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Loss: {running_loss/len(train_loader):.4f}')

    # Evaluation
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_targets, all_preds) * 100
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')

    # Print results
    print(f'Fold {fold+1} Accuracy: {accuracy:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')




Using device: cuda:0

----- Fold 1 -----


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 75.0MB/s]
Epoch 1/30: 100%|██████████| 4/4 [00:03<00:00,  1.22it/s]


Loss: 1.2669


Epoch 2/30: 100%|██████████| 4/4 [00:02<00:00,  1.88it/s]


Loss: 0.6308


...

Loss: 0.0194


Epoch 29/30: 100%|██████████| 4/4 [00:02<00:00,  1.79it/s]


Loss: 0.0148


Epoch 30/30: 100%|██████████| 4/4 [00:02<00:00,  1.79it/s]


Loss: 0.0141
Fold 3 Accuracy: 96.23%
Precision: 0.9642, Recall: 0.9623, F1-score: 0.9623


In [ ]:
# prompt: do the above same with same hyperparameters for resnet for the whole code, write the code even if it is same as above

# ... (Your existing code)

# Define device (GPU if available)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Hyperparameters
batch_size = 64
num_epochs = 30
learning_rate = 1e-4

# 5-fold cross-validation
kfold = KFold(n_splits=3, shuffle=True, random_state=42)

for fold, (train_ids, test_ids) in enumerate(kfold.split(all_images)):
    print(f"\n----- Fold {fold+1} -----")

    # Sample data for current fold
    train_images = torch.stack([all_images[i] for i in train_ids])
    train_labels = torch.tensor([all_labels[i] for i in train_ids])
    test_images = torch.stack([all_images[i] for i in test_ids])
    test_labels = torch.tensor([all_labels[i] for i in test_ids])

    # Create data loaders
    train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = torch.utils.data.TensorDataset(test_images, test_labels)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize ResNet-34 model
    model = models.resnet34(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 4)  # Modify classifier for 4 classes
    model = model.to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # Adam optimizer

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f'Loss: {running_loss/len(train_loader):.4f}')

    # Evaluation
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_targets, all_preds) * 100
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average='weighted')

    # Print results
    print(f'Fold {fold+1} Accuracy: {accuracy:.2f}%')
    print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}')


Using device: cuda:0

----- Fold 1 -----


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/30: 100%|██████████| 4/4 [00:01<00:00,  3.39it/s]


Loss: 1.1006


Epoch 2/30: 100%|██████████| 4/4 [00:01<00:00,  3.55it/s]


Loss: 0.1920


...

Loss: 0.0594


Epoch 29/30: 100%|██████████| 4/4 [00:01<00:00,  3.46it/s]


Loss: 0.0172


Epoch 30/30: 100%|██████████| 4/4 [00:01<00:00,  3.50it/s]


Loss: 0.0231
Fold 3 Accuracy: 93.40%
Precision: 0.9385, Recall: 0.9340, F1-score: 0.9333
